# Importing necessities
 

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
import cv2
import os
import posenet
import time
import pandas as pd
import numpy as np
import pickle


# Loading Model Names

In [ ]:
filename_RFC = "RFC.pickle"
filename_ANN = "ANN"
filename_CNN = "CNN"
filename_RNN = "RNN"
filename_LSTM = "LSTM"

### Reading One Hot Encoding categories

In [ ]:
with open("OneHotEncoding categories.txt", 'r') as f:
    ohe_categories = f.readline()
ohe_categories = list(ohe_categories.split("'")[1:-1])
while " " in ohe_categories:
    ohe_categories.remove(" ")
print(ohe_categories)

 **Configuring my cudnn for tensorflow 1** <br><br>
 **Needed for my laptop, unsure for others**

In [ ]:
global sess
global graph

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

from tensorflow.python.keras.backend import set_session
from tensorflow.python.keras.models import load_model

graph = tf.get_default_graph()

set_session(sess)

#model_RFC = pickle.load(open(filename_RFC, 'rb'))
model_ANN = tf.keras.models.load_model(filename_ANN)
model_CNN = tf.keras.models.load_model(filename_CNN)
model_RNN = tf.keras.models.load_model(filename_RNN)
model_LSTM = tf.keras.models.load_model(filename_LSTM)

In [ ]:
#session = tf.keras.backend.get_session()
#init = tf.global_variables_initializer()
#session.run(init)

# Prediction

In [ ]:
#physical_devices = tf.config.experimental.list_physical_devices('GPU')
#assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
#config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
temporary = pd.DataFrame()
seconds=2


#with tf.Session() as sess:
with graph.as_default():
    
    set_session(sess)
        
    # Setting ouput stride to 16
    output_stride = 16
    model_id = 101
    model_cfg, model_outputs = posenet.load_model(model_id, sess)

    #cap = cv2.VideoCapture("E:\\AIML community\\Deep Learning Fall Detection\\Final Project\\data\\Online_fall.mp4")
    cap = cv2.VideoCapture(0)
    input_image, display_image, output_scale = posenet.read_cap(cap)
    
    #file_save = "live"
    #result = cv2.VideoWriter(f'{file_save}.avi',  
    #                     cv2.VideoWriter_fourcc(*'MJPG'), 
    #                     10, (int(cap.get(3)), int(cap.get(4)))) 

    start = time.time()
    while True:
        input_image, display_image, output_scale = posenet.read_cap(cap)

        heatmaps_result, offsets_result, displacement_fwd_result, displacement_bwd_result = sess.run(
            model_outputs,
            feed_dict={'image:0': input_image})

        pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multi.decode_multiple_poses(
            heatmaps_result.squeeze(axis=0),
            offsets_result.squeeze(axis=0),
            displacement_fwd_result.squeeze(axis=0),
            displacement_bwd_result.squeeze(axis=0),
            output_stride=output_stride,
            max_pose_detections=10,
            min_pose_score=0.15)

        keypoint_coords *= output_scale

        # TODO this isn't particularly fast, use GL for drawing and display someday...
        overlay_image = posenet.draw_skel_and_kp(
            display_image, pose_scores, keypoint_scores, keypoint_coords,
            min_pose_score=0.15, min_part_score=0.3)
        # setting min_pose_score and min_part_score =0 prints the estimated value

        x = keypoint_coords[0][:,0]
        y = keypoint_coords[0][:,1]

        data = []
        for i in range(len(posenet.PART_NAMES)):
            for j in [x, y]:
                data.append(j[i])
        #print(temp)

        temp = pd.DataFrame(data).T

        prediction_RFC = model_RFC.predict(temp)
        prediction_ANN = ohe_categories[np.argmax(model_ANN.predict(temp))]
        prediction_CNN = "Not Ready yet"
        prediction_RNN = "Not Ready yet"
        prediction_LSTM = "Not Ready yet"
        
        temporary = pd.concat([temporary, temp], ignore_index=True)
        X_CNN = []
        if len(temporary) == seconds*30:
            X_CNN.append(temporary.values)
            X_CNN = np.asarray(X_CNN).reshape(-1, seconds*30, 34, 1)
            prediction_CNN = ohe_categories[np.argmax(model_CNN.predict(X_CNN))]
            X_RNN = np.asarray(X_CNN).reshape(-1, seconds*30, 34)
            prediction_RNN = ohe_categories[np.argmax(model_RNN.predict(X_RNN))]
            prediction_LSTM = ohe_categories[np.argmax(model_LSTM.predict(X_RNN))]
            temporary.drop(temporary.index[:1], inplace=True)
            
        
        font = cv2.FONT_HERSHEY_SIMPLEX 
        fontscale = 1
        color = (255, 0, 255)   # BGR
        thickness = 1
        
        #coords_RFC = (0, 20)
        coords_ANN = (0, 20)
        coords_CNN = (0, 50)
        coords_RNN = (250, 20)
        coords_LSTM = (250, 50)
        
        #cv2.putText(overlay_image, 'RFC = '+prediction_RFC[0], coords_RFC, font, fontscale, color, thickness)
        cv2.putText(overlay_image, 'RNN = '+prediction_RNN, coords_RNN, font, fontscale, color, thickness)
        cv2.putText(overlay_image, 'ANN = '+prediction_ANN, coords_ANN, font, fontscale, color, thickness)
        cv2.putText(overlay_image, 'CNN = '+prediction_CNN, coords_CNN, font, fontscale, color, thickness)
        cv2.putText(overlay_image, 'LSTM = '+prediction_LSTM, coords_LSTM, font, fontscale, color, thickness)
        
        cv2.imshow('posenet', overlay_image)
        
        #result.write(overlay_image)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            #result.release()
            break

# print(cap.get(cv2.CAP_PROP_FPS)) -> frame rate
print(f'time taken = {(time.time() - start)/60} mins')


In [ ]:
model = tf.keras.models.load_model('ANN')
temp = [258.8809064423327, 1096.1560963877098, 247.6743717696233,
       1103.16589102719, 237.49869866443905, 1111.8137990339578,
       211.85583551118512, 1091.1299527575502, 197.20654406924518,
       1082.078272799418, 240.85130642588032, 1042.566650771797,
       224.6896357542929, 1012.5111997844092, 290.9416211163948,
       1054.6050492941076, 324.71345306269507, 967.8415862775054,
       305.6517183036645, 1068.0997568885393, 379.74106977782867,
       972.2731288460247, 355.97467642055307, 1028.445288295582,
       314.87228964965647, 906.4956276720152, 445.0320768951543,
       1030.3417711458944, 406.206993282916, 952.7757588370903,
       535.5605948946843, 1017.1907375791313, 486.6165264304237,
       951.2660654441721]

temp = pd.DataFrame(temp).T
x = model.predict(temp)

print(ohe_categories[np.argmax(x)])